<a href="https://colab.research.google.com/github/ENV716/Energy_Modeling_F2022/blob/main/Lab03/Lab03_PivotTables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 03 – More on Filtering Data Sets and Pivot Tables in Python**

Learning goals for Lab 3:

*   Learn how to change column type from character to float/int;
*   Learn how to create pivot tables from a data frame using function pivot_table();
*   Learn how to save your table as an excel file.



If you have not mounted google drive yet please do so by running the code chunk below. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set up working directory.

In [3]:
 import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Before we get started take a look at the Runtime tab and all the ways you can run your notebook code chunks.


*   Run all
*   Run before
*   Run selection
*   Run after


# Import Excel File

Before building the pivot table you need to import the data. Here we will import sheet GEN20 from e-grid to Python.

In [4]:
import pandas as pd
import numpy as np

In [16]:
egrid=pd.read_excel("egrid2020_data.xlsx",sheet_name="GEN20",skiprows=1,keep_default_na=False)
egrid.head()

,SEQGEN20,YEAR,PSTATABB,PNAME,ORISPL,GENID,NUMBLR,GENSTAT,PRMVR,FUELG1,NAMEPCAP,CFACT,GENNTAN,GENNTOZ,GENERSRC,GENYRONL,GENYRRET
0,1,2020,AK,7-Mile Ridge Wind Project,60814,WT1,0,CN,WT,WND,1.8,,,,,,
1,2,2020,AK,Agrium Kenai Nitrogen Operations,54452,744A,0,OS,GT,NG,2.5,,,,,1977,
2,3,2020,AK,Agrium Kenai Nitrogen Operations,54452,744B,0,OS,GT,NG,2.5,,,,,1977,
3,4,2020,AK,Agrium Kenai Nitrogen Operations,54452,744C,0,OS,GT,NG,2.5,,,,,1977,
4,5,2020,AK,Agrium Kenai Nitrogen Operations,54452,744D,0,OS,GT,NG,2.5,,,,,1977,


In [17]:
egrid.columns

Index(['SEQGEN20', 'YEAR', 'PSTATABB', 'PNAME', 'ORISPL', 'GENID', 'NUMBLR',
       'GENSTAT', 'PRMVR', 'FUELG1', 'NAMEPCAP', 'CFACT', 'GENNTAN', 'GENNTOZ',
       'GENERSRC', 'GENYRONL', 'GENYRRET'],
      dtype='object')

# Building a Pivot Table with Python

A pivot table allows you to reshape your original table or data frame. Pandas offers two functions to pivot tables: pivot() and pivot_table(). The pivot_table() is a generalization of pivot() that allows you to aggregate multiple values and create summaries. 

The pivot_table() function takes two main arguments: index and columns. The index will identify which column of the data frame you want to be represented in the rows of the pivot table. If you think about the pivot table in Excel, the elements will put in the Rows category should be the same as you put in index. Columns is whichever element from the data frame you want to appear in the columns of the pivot. 

The documentation for pivot_table() can be found at:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html.

Let’s see how to use pivot_table(). 


## Creating a subset of egrid table

Imagine you want to filter only generators online between 2007 and 2018. Let’s create a subset of generators that satisfy that criteria before entering pivot_table().

In [18]:
egrid.dtypes

SEQGEN20     int64
YEAR         int64
PSTATABB    object
PNAME       object
ORISPL       int64
GENID       object
NUMBLR       int64
GENSTAT     object
PRMVR       object
FUELG1      object
NAMEPCAP    object
CFACT       object
GENNTAN     object
GENNTOZ     object
GENERSRC    object
GENYRONL    object
GENYRRET    object
dtype: object

Note GENYRONL is an object so we need to convert to numeric using pd.to_numeric() function.

In [19]:
#Convert column GENYRONL to numeric
egrid['GENYRONL']=pd.to_numeric(egrid['GENYRONL'])

In [20]:
#Now that column GENYRONL is numeric we can use if statement to filter generator online between 2007 and 2018 and create a subset of egrid
sub_egrid=egrid[(egrid.GENYRONL >= 2007) & (egrid.GENYRONL <= 2018)]

In [21]:
#Creating a another subset with only the 5 columns of interest
sub_egrid=sub_egrid.loc[:,['PSTATABB','PNAME','FUELG1','CFACT','GENYRONL']]
sub_egrid.head()

,PSTATABB,PNAME,FUELG1,CFACT,GENYRONL
13,AK,Alakanuk,DFO,0,2013.0
18,AK,Allison Creek Hydro,WAT,0.365,2016.0
23,AK,Anchorage 1,NG,0.011,2007.0
25,AK,Anchorage 1,DFO,0,2012.0
31,AK,Angoon,DFO,0.125,2009.0


If you want to fix the decimal points on the GENYRONL columns we can use the to_numeric() conversion again, but specify you want the column to be converted to integer using argument downcast=’signed’. You can use ’signed’ or ‘integer’. 

In [41]:
sub_egrid['GENYRONL']=pd.to_numeric(sub_egrid['GENYRONL'],downcast='integer') 
sub_egrid.head()

,PSTATABB,PNAME,FUELG1,CFACT,GENYRONL
13,AK,Alakanuk,DFO,0.000,2013
18,AK,Allison Creek Hydro,WAT,0.365,2016
23,AK,Anchorage 1,NG,0.011,2007
25,AK,Anchorage 1,DFO,0.000,2012
31,AK,Angoon,DFO,0.125,2009


## Creating a pivot table with the subset/filtered egrid table

To create a pivot table you need to use the pivot_table() function. If you want states in the rows specify index= ‘PSTATABB’ and to get year in the columns specify columns=‘GENYRONL’. 
Since we want to count units that come online each year specify aggfunc=”count”.You can include another argument that says values=’PNAMES’ so the function nows what to count. It could be any column on sub_df assigned to values. We are just making sure the counting only happens once.

In [42]:
pivot1=sub_egrid.pivot_table(values="PNAME",index="PSTATABB",columns="GENYRONL",aggfunc="count",margins=True)
pivot1.head(10)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
PSTATABB,,,,,,,,,,,,,
AK,12.0,14.0,13.0,20.0,5.0,20.0,20.0,14.0,23.0,12.0,14.0,3.0,170
AL,1.0,3.0,NaN,6.0,2.0,NaN,3.0,1.0,NaN,1.0,6.0,1.0,24
AR,10.0,NaN,1.0,8.0,NaN,1.0,1.0,NaN,1.0,2.0,3.0,1.0,28
AZ,NaN,6.0,7.0,5.0,33.0,30.0,20.0,8.0,7.0,12.0,11.0,6.0,145
CA,30.0,41.0,45.0,69.0,127.0,164.0,170.0,182.0,180.0,131.0,127.0,90.0,1356
CO,12.0,21.0,9.0,7.0,6.0,18.0,13.0,8.0,14.0,12.0,17.0,16.0,153
CT,3.0,9.0,6.0,23.0,9.0,7.0,4.0,11.0,5.0,16.0,6.0,26.0,125
DC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,1.0,5
DE,NaN,NaN,NaN,2.0,4.0,4.0,8.0,2.0,3.0,NaN,1.0,2.0,26


If you want to fix the NaN, you can add the fill_value argument to the function as bellow.

In [43]:
pivot1=sub_egrid.pivot_table(values="PNAME",index="PSTATABB",columns="GENYRONL",aggfunc="count",fill_value="",margins=True)
pivot1.head(51)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
PSTATABB,,,,,,,,,,,,,
AK,12.0,14.0,13.0,20.0,5.0,20.0,20.0,14.0,23.0,12.0,14.0,3.0,170
AL,1.0,3.0,,6.0,2.0,,3.0,1.0,,1.0,6.0,1.0,24
AR,10.0,,1.0,8.0,,1.0,1.0,,1.0,2.0,3.0,1.0,28
AZ,,6.0,7.0,5.0,33.0,30.0,20.0,8.0,7.0,12.0,11.0,6.0,145
CA,30.0,41.0,45.0,69.0,127.0,164.0,170.0,182.0,180.0,131.0,127.0,90.0,1356
CO,12.0,21.0,9.0,7.0,6.0,18.0,13.0,8.0,14.0,12.0,17.0,16.0,153
CT,3.0,9.0,6.0,23.0,9.0,7.0,4.0,11.0,5.0,16.0,6.0,26.0,125
DC,,,,,,,,,3.0,1.0,,1.0,5
DE,,,,2.0,4.0,4.0,8.0,2.0,3.0,,1.0,2.0,26


If you want to change the name of the summary columns you can add another argument margins_name="Total".

In [44]:
pivot1=sub_egrid.pivot_table(values="PNAME",index="PSTATABB",columns="GENYRONL",aggfunc="count",margins_name="Total",fill_value="",margins=True)
pivot1.head(10)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Total
PSTATABB,,,,,,,,,,,,,
AK,12.0,14.0,13.0,20.0,5.0,20.0,20.0,14.0,23.0,12.0,14.0,3.0,170
AL,1.0,3.0,,6.0,2.0,,3.0,1.0,,1.0,6.0,1.0,24
AR,10.0,,1.0,8.0,,1.0,1.0,,1.0,2.0,3.0,1.0,28
AZ,,6.0,7.0,5.0,33.0,30.0,20.0,8.0,7.0,12.0,11.0,6.0,145
CA,30.0,41.0,45.0,69.0,127.0,164.0,170.0,182.0,180.0,131.0,127.0,90.0,1356
CO,12.0,21.0,9.0,7.0,6.0,18.0,13.0,8.0,14.0,12.0,17.0,16.0,153
CT,3.0,9.0,6.0,23.0,9.0,7.0,4.0,11.0,5.0,16.0,6.0,26.0,125
DC,,,,,,,,,3.0,1.0,,1.0,5
DE,,,,2.0,4.0,4.0,8.0,2.0,3.0,,1.0,2.0,26


## Saving you pivot table in Excel

You can export your pivot table to Excel using the command .to_excel() as below. And a excel file will be created on your working directory. If you want to specify a name for the sheet use teh argumento sheet_name.

In [45]:
pivot1.to_excel("output1.xlsx",sheet_name="PIVOT1")

Where was you file stored?? Since you did not specify the path, the code you store it at your current working directory.

One of the confusing points with the pivot_table() is the use of columns and values. Remember, columns provide a way to segment the values you care about. The aggregation functions are applied to the values you list. Let’s see a couple more examples.

## More pivot tables examples

Let’s create a table that shows average capacity factor by state of the generators that came online in 2007, 2008, …, 2018. You need to change values to ‘CFACT’ and aggfunc to “mean”.

In [46]:
sub_egrid.dtypes

PSTATABB     object
PNAME        object
FUELG1       object
CFACT       float64
GENYRONL      int16
dtype: object

In [47]:
sub_egrid['CFACT']=pd.to_numeric(sub_egrid['CFACT'])
sub_egrid.dtypes

PSTATABB     object
PNAME        object
FUELG1       object
CFACT       float64
GENYRONL      int16
dtype: object

In [48]:
pivot2=sub_egrid.pivot_table(values="CFACT",index="PSTATABB",columns="GENYRONL",aggfunc="mean",margins_name="Av CFAC",fill_value="",margins=True)
pivot2.head(10)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Av CFAC
PSTATABB,,,,,,,,,,,,,
AK,0.240667,0.166571,0.2,0.140529,0.0856,0.17425,0.2415,0.2118,0.280591,0.143583,0.138692,0.159,0.194075
AL,0.67,0.743333,,0.531833,0.6005,,0.581,0.759,,0.199,0.2155,0.227,0.479708
AR,0.2312,,0.405,0.653375,,0.66,0.007,,0.23,0.52,0.350667,0.223,0.398429
AZ,,0.243,0.201429,0.1782,0.165121,0.2902,0.28245,0.232625,0.286,0.27725,0.204091,0.164833,0.234393
CA,0.149267,0.334325,0.273614,0.323866,0.139206,0.302366,0.248606,0.279203,0.192833,0.295031,0.33532,0.333807,0.264536
CO,0.227083,0.16781,0.204,0.209857,0.346,0.407667,0.456769,0.313,0.370357,0.273917,0.274824,0.2255,0.288641
CT,0.205,0.351625,0.539,0.035043,0.268,0.272,0.553333,0.181364,0.6672,0.312,0.129667,0.302615,0.263123
DC,,,,,,,,,0.451,,,0.862,0.55375
DE,,,,0.4915,0.07925,0.52325,0.68825,0.678,0.364667,,0.189,0.18,0.457615


Now let’s create a table that count generators again but let’s include another layer of discretization, unit primary fuel. Add another element ‘FUELG1’ to the index argument as show below.



In [50]:
pivot3=sub_egrid.pivot_table(values="PNAME",index=["PSTATABB","FUELG1"],columns="GENYRONL",aggfunc="count",margins_name="Total",fill_value="",margins=True)
pivot3.head(10)

GENYRONL        2007 2008 2009  2010 2011  2012 2013 2014  2015  2016  2017  \
PSTATABB FUELG1                                                               
AK       DFO     9.0  9.0  9.0  20.0  3.0  11.0  9.0  7.0   8.0  11.0  11.0   
         LFG                                4.0  1.0                          
         MWH                                1.0             2.0               
         NG      1.0  3.0  1.0                   6.0  3.0  10.0         3.0   
         WAT          1.0  1.0                        4.0   2.0   1.0         
         WND          1.0  2.0        2.0   4.0  4.0        1.0               
         WO      2.0                                                          
AL       BLQ     1.0                                  1.0               1.0   
         LFG                     1.0  1.0        3.0                          
         MWH                                                            1.0   

GENYRONL        2018  Total  
PSTATABB FUELG1              
AK       DFO     3.0    110  
         LFG              5  
         MWH              3  
         NG              27  
         WAT              9  
         WND             14  
         WO               2  
AL       BLQ              3  
         LFG              5  
         MWH              1

# Exercise H7 - 1

Use the Pandas library in Python to open the PLNT2020 spreadsheet from eGRID20. <br>
 Present a table that shows for each state the number of power plants of different fuel types.

In [51]:
egrid=pd.read_excel("egrid2020_data.xlsx",sheet_name="PLNT20",skiprows=1,keep_default_na=False)

pivot1=pd.pivot_table(egrid,index="PSTATABB",values="PNAME",columns="PLFUELCT",aggfunc='count')
pivot1.head(10)

PLFUELCT,,BIOMASS,COAL,GAS,GEOTHERMAL,HYDRO,NUCLEAR,OFSL,OIL,OTHF,SOLAR,WIND
PSTATABB,,,,,,,,,,,,
AK,NaN,1.0,5.0,14.0,NaN,33.0,NaN,NaN,91.0,4.0,NaN,6.0
AL,NaN,12.0,6.0,28.0,NaN,25.0,2.0,1.0,3.0,1.0,11.0,NaN
AR,2.0,8.0,5.0,24.0,NaN,19.0,1.0,NaN,3.0,NaN,16.0,NaN
AZ,4.0,6.0,4.0,33.0,NaN,12.0,1.0,NaN,2.0,4.0,92.0,6.0
CA,30.0,114.0,1.0,347.0,45.0,264.0,2.0,4.0,20.0,87.0,724.0,135.0
CO,3.0,4.0,11.0,28.0,NaN,50.0,NaN,1.0,11.0,4.0,122.0,34.0
CT,2.0,7.0,NaN,50.0,NaN,15.0,1.0,NaN,23.0,NaN,53.0,1.0
DC,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN
DE,1.0,3.0,1.0,8.0,NaN,NaN,NaN,NaN,5.0,2.0,13.0,1.0


# Exercise H7 - 2

Present a table that shows for each state the total installed capacity of all different fuel types. Leave blank the cells for which there is not a value

In [52]:
egrid.NAMEPCAP=pd.to_numeric(egrid.NAMEPCAP)
pivot2=pd.pivot_table(egrid,index="PSTATABB",values="NAMEPCAP",columns="PLFUELCT",aggfunc='sum',fill_value="")
pivot2.head(10)

PLFUELCT,,BIOMASS,COAL,GAS,GEOTHERMAL,HYDRO,NUCLEAR,OFSL,OIL,OTHF,SOLAR,WIND
PSTATABB,,,,,,,,,,,,
AK,,11.5,217.2,1731.4,,535.4,,,933.0,46.0,,59.3
AL,,693.6,13121.5,17143.5,,3337.3,5270.4,3.8,31.6,1.6,770.0,
AR,341.0,301.9,5487.0,10192.9,,1350.9,1845.0,,20.9,,728.7,
AZ,1763.0,42.4,6539.1,22573.8,,2916.1,4209.6,,29.1,72.0,3830.2,887.8
CA,9353.4,1468.5,62.5,55284.2,3587.7,13866.4,4577.0,642.3,883.6,2540.4,19121.0,6963.7
CO,875.0,35.6,6137.5,7941.5,,1211.4,,3.0,221.4,18.8,1811.3,5289.6
CT,140.1,254.6,,7584.6,,217.2,2162.9,,2387.2,,308.2,5.0
DC,,14.1,,22.9,,,,,,,15.5,
DE,30.0,14.2,804.1,800.2,,,,,1993.8,333.2,202.9,2.0


# Exercise H7 - 3

Present a table that summarizes for each state info on coal, gas, and nuclear plants, showing the size of the smallest plant, the largest plant, the average size, and the total installed capacity



In [54]:
egrid_sub=egrid[(egrid.PLFUELCT=="COAL")|(egrid.PLFUELCT=="GAS")|(egrid.PLFUELCT=="NUCLEAR")]
pivot3=pd.pivot_table(egrid_sub,index="PSTATABB",values=["NAMEPCAP"],columns="PLFUELCT",aggfunc=[min,max,np.mean,sum],fill_value="")
pivot3.head(10)

min                   max                         mean  \
         NAMEPCAP              NAMEPCAP                     NAMEPCAP   
PLFUELCT     COAL  GAS NUCLEAR     COAL     GAS NUCLEAR         COAL   
PSTATABB                                                               
AK           22.5  8.4             92.8   418.0                43.44   
AL         1264.7  3.4  1776.4   3615.5  2683.8  3494.0  2186.916667   
AR          558.0  4.0  1845.0   1800.0  2428.0  1845.0       1097.4   
AZ          821.8  3.1  4209.6   2409.3  1857.0  4209.6     1634.775   
CA           62.5  1.0  2254.0     62.5  3102.0  2323.0         62.5   
CO           43.7  3.0           1635.3  1426.9           557.954545   
CT                 1.0  2162.9           1157.9  2162.9                
DC                 4.6                     10.8                        
DE          804.1  1.3            804.1   361.0                804.1   
FL           98.0  5.8  2160.0   5303.7  6071.5  3678.7  1764.357143   

                                       sum                   
                                  NAMEPCAP                   
PLFUELCT         GAS      NUCLEAR     COAL      GAS NUCLEAR  
PSTATABB                                                     
AK        123.671429                 217.2   1731.4          
AL        612.267857       2635.2  13121.5  17143.5  5270.4  
AR        424.704167       1845.0   5487.0  10192.9  1845.0  
AZ        684.054545       4209.6   6539.1  22573.8  4209.6  
CA        159.320461       2288.5     62.5  55284.2  4577.0  
CO           283.625                6137.5   7941.5          
CT           151.692       2162.9            7584.6  2162.9  
DC          7.633333                           22.9          
DE           100.025                 804.1    800.2          
FL        964.207692  2779.566667  12350.5  75208.2  8338.7

# Exercise H7 - 4

What is the average capacity factor and average nameplate capacity of coal, gas and nuclear plants in each state?

In [ ]:
#Convert column CAPFAC to numeric
#Your code
#Create pivot table

# Exercise H7 - 5

Repeat 4, but this time show for each state the total nameplate capacity and the average CF for gas, nuclear, and coal plants

In [ ]:
# Your code

# Exercise H7 - 6

Repeat 5 but now show the average CF for each fuel for all the country, and the total installed capacity for each fuel (coal, gas, nuke) for all the country

In [ ]:
# Your code